# 2D Spot Detection with Icy

## Discipline: Spot / object counting (SptCnt)¶

Task: Estimate the number of objects

This project illustrates the 2D counting of small vesicle like objects.

## The input images

The images were generated by SIMCEP, a widefield fluorescence microscopy biological images simulator.

First we create the folders needed by the workflow and download the dataset from zenodo.

In [ ]:
IN_FOLDER = "/home/jovyan/data/in/"
GT_FOLDER = "/home/jovyan/data/gt/"
OUT_FOLDER = "/home/jovyan/data/out/"

!mkdir -p $IN_FOLDER
!mkdir -p $GT_FOLDER
!mkdir -p $OUT_FOLDER

We download the input and ground truth images from zenodo and move them into the ``in`` and ``gt`` folder.

In [ ]:
!cd $GT_FOLDER && wget -np -nd -r -l1 -A_lbl.tif https://zenodo.org/record/3549883/
!cd $IN_FOLDER && wget -np -nd -r -l1 -ASpt.tif https://zenodo.org/record/3549883/ 

We rename the ground truth images, so that they have the same names as the input images.

In [ ]:
!cd $GT_FOLDER && rename.ul '_lbl' '' *.tif

We display the input images. If after execution of the cell the images do not show, run the cell a second time.

In [ ]:
import os
import tifffile
import matplotlib.pyplot
files = [i.path for i in os.scandir(IN_FOLDER) if i.is_file()]
in_paths_sources = [i for i in files if ('.tif' in i) ]
sources = []
for path in in_paths_sources:
    source = tifffile.imread(path)
    tifffile.imshow(source, cmap='gray')
    sources.append(source)

## The image analysis workflow

The image analysis workflow is implemented as an Icy-protocol. You can open it with Icy from the protocol editor, by loading the file [protocol.protocol](https://raw.githubusercontent.com/MontpellierRessourcesImagerie/Jupyter_SpotDetection-Icy/master/protocol.protocol). It uses the [Wavelet Spot Detector Block](http://icy.bioimageanalysis.org/plugin/wavelet-spot-detector-block/).

![spot_detection_protocol_icy.png](https://raw.githubusercontent.com/MontpellierRessourcesImagerie/Jupyter_SpotDetection-Icy/master/spot_detection_protocol_icy.png)

## Running the workflow

The workflow has one parameter the ``icy_scale3sensitivity`` with the default value 100.

In [ ]:
SCALE_3_SENSITIVITY = 100

We run the workflow, using the ``--local`` switch, so that the server will not be used.

In [ ]:
!python /app/wrapper.py  --icy_scale3sensitivity $SCALE_3_SENSITIVITY --infolder $IN_FOLDER --gtfolder $GT_FOLDER --outfolder $OUT_FOLDER --local

## Displaying the results

We display the detected spots as red circles on the input images and count the number of spots per image.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.measure import label, regionprops
from skimage.color import label2rgb

files = [i.path for i in os.scandir(OUT_FOLDER) if i.is_file()]
paths_sources = [i for i in files if ('.tif' in i) ]
pointImages = []
index = 0
nrOfSpots = []

In [ ]:
for path in paths_sources:
    fig, ax = plt.subplots(figsize=(20, 12))
    pointImage = tifffile.imread(path)
    label_image = label(pointImage)
    image_label_overlay = label2rgb(label_image, image=sources[index])
    ax.imshow(sources[index], cmap="gray")
    spotCount = 0
    for region in regionprops(label_image):
        (x,y) = region.centroid
        radius = region.equivalent_diameter/2.0
        rect = mpatches.Circle((y,x), 5, fill=False, edgecolor='red')
        ax.add_patch(rect)
        spotCount = spotCount + 1
    nrOfSpots.append(spotCount)
        
    index = index + 1
    plt.tight_layout()
    plt.show()
    print("number of spots: " + str(spotCount))

Finally we display the number of spots for each image:

In [ ]:
import statistics
index = 0
for path in in_paths_sources:
    print(path + "\t" + str(nrOfSpots[index]))
    index = index + 1
    
print("Mean" + "\t" + str(statistics.mean(nrOfSpots)))
print("StdDev" + "\t" + str(statistics.stdev(nrOfSpots)))

And we calculate and display a boxplot of the distribution of the number of cells per image.

In [ ]:
plt.boxplot(nrOfSpots)
plt.show()